In [1]:
from bs4 import BeautifulSoup, Comment 
import requests
import time, os, sys
import re
import lxml
from lxml.html.soupparser import fromstring
import unicodedata

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
def strip_accents_and_periods(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text).replace('.','')


def update_row_with_dict(df,d,idx):
    for key in d.keys():
        df.loc[idx, key] = d.get(key)

In [3]:
def scrape_player(playerurl, fa_year):
    d = {}
    prev_season = str(fa_year-1) + '-' + str(fa_year-2000)

    playerresponse = requests.get(playerurl)
    playerpage = playerresponse.text
    playersoup = BeautifulSoup(playerpage, "lxml")

    # Get height
    try:
        weight = playersoup.find('span',{'itemprop': 'weight'}).getText().replace('lb','')
    except:
        weight = np.nan
    d['weight']= weight

    # Get weight
    try:
        height = playersoup.find('span',{'itemprop': 'height'}).getText()
#       height_ft  = height.split('-')[0]) + float(height.split('-')[1])/12
    except:
        height = np.nan
    d['height']= height

    
    # Get Basic Stats:  G, GS, MP, FG%, 3P, 3P%, FT, FT%, ORB, TRB, AST, STL, BLK, TOV, PTS
    
    dfpergame = pd.read_html(str(playersoup.find(id='per_game')))[0]
    dfpergame = dfpergame.drop_duplicates(subset=['Season'])
    dfpergame = dfpergame.set_index('Season')
#     display(dfpergame)

    featurelist = ['Age', 'G', 'GS', 'MP', 'FG%', '3P', '3P%', 'FT', 'FT%', 'ORB', 'TRB', 
                    'AST', 'STL', 'BLK', 'TOV', 'PTS']
    
    for feature in featurelist:
        try: 
            d[feature] = dfpergame.loc[prev_season, feature] 
        except: 
            d[feature] = np.nan


    # Get advanced stats

    dfadvanced = pd.read_html(str(playersoup.find(id='advanced')))[0]
    dfadvanced = dfadvanced.drop_duplicates(subset=['Season'])
    dfadvanced = dfadvanced.set_index('Season')
#     display(dfadvanced)
    
    featurelist = ['USG%', 'TS%']
    
    for feature in featurelist:
        try: 
            d[feature] = dfadvanced.loc[prev_season, feature] 
        except: 
            d[feature] = np.nan

    return(d)

In [4]:
playerurl = 'https://www.basketball-reference.com/players/d/duranke01.html'
season = 2016
d = scrape_player(playerurl, season)
d

{'weight': '240',
 'height': '6-10',
 'Age': 27.0,
 'G': '72',
 'GS': '72',
 'MP': '35.8',
 'FG%': '.505',
 '3P': '2.6',
 '3P%': '.387',
 'FT': '6.2',
 'FT%': '.898',
 'ORB': '0.6',
 'TRB': '8.2',
 'AST': '5.0',
 'STL': '1.0',
 'BLK': '1.2',
 'TOV': '3.5',
 'PTS': '28.2',
 'USG%': 30.6,
 'TS%': 0.634}

In [5]:
def scrape_fa_year(fa_year):
    
    FA_url = 'https://www.basketball-reference.com/friv/free_agents.cgi?year=' + str(fa_year) 
    prev_season = str(fa_year-1) + '-' + str(fa_year-2000)

    response = requests.get(FA_url)
    page = response.text
    FAsoup = BeautifulSoup(page, "lxml")

    table = FAsoup.find('table')
    rows = [row for row in table.find_all('tr')]  # tr tag is for rows
    rows_data = [[td.getText() for td in rows[i].findAll('td')]
                    for i in range(len(rows))]
        
    df = pd.DataFrame()
    for i in range(1,len(rows)):
        try:
            name_year = (rows_data[i][0] + ' ' + str(fa_year)).replace(' ','_')
            name_year = strip_accents_and_periods(name_year)
        
            name = (rows_data[i][0]).replace(' ','-')
            name = strip_accents_and_periods(name)
        
            pos = rows_data[i][1]
            d = {'pos': pos, 'name':name}
            update_row_with_dict(df,d,name_year)
        
            playerurl = 'https://www.basketball-reference.com' + rows[i].find_all('td')[0].find('a')['href']
            playerdict = scrape_player(playerurl, fa_year)
            update_row_with_dict(df,playerdict,name_year)
        except:
            continue
            
    return df

In [6]:
df2019 = scrape_fa_year(2019)
df2019.head(20)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,pos,name,weight,height,Age,G,GS,MP,FG%,3P,...,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS,USG%,TS%
Kevin_Durant_2019,F-G,Kevin-Durant,240,6-10,30.0,78,78,34.6,.521,1.8,...,.885,0.4,6.4,5.9,0.7,1.1,2.9,26.0,29.0,0.631
Nikola_Vucevic_2019,C,Nikola-Vucevic,260,6-11,28.0,80.0,80.0,31.4,0.518,1.1,...,0.789,2.8,12.0,3.8,1.0,1.1,2.0,20.8,28.0,0.573
Kawhi_Leonard_2019,F,Kawhi-Leonard,225,6-7,27.0,60.0,60.0,34.0,0.496,1.9,...,0.854,1.3,7.3,3.3,1.8,0.4,2.0,26.6,30.3,0.606
Kyrie_Irving_2019,G,Kyrie-Irving,195,6-2,26.0,67.0,67.0,33.0,0.487,2.6,...,0.873,1.1,5.0,6.9,1.5,0.5,2.6,23.8,29.6,0.592
Jimmy_Butler_2019,F-G,Jimmy-Butler,230,6-7,29.0,65.0,65.0,33.6,0.462,1.0,...,0.855,1.9,5.3,4.0,1.9,0.6,1.5,18.7,22.3,0.571
Al_Horford_2019,C-F,Al-Horford,240,6-9,32.0,68.0,68.0,29.0,0.535,1.1,...,0.821,1.8,6.7,4.2,0.9,1.3,1.5,13.6,18.9,0.605
Kemba_Walker_2019,G,Kemba-Walker,184,6-0,28.0,82.0,82.0,34.9,0.434,3.2,...,0.844,0.6,4.4,5.9,1.2,0.4,2.6,25.6,31.5,0.558
Tobias_Harris_2019,F,Tobias-Harris,226,6-8,26.0,82.0,82.0,34.7,0.487,1.9,...,0.866,0.8,7.9,2.8,0.6,0.5,1.8,20.0,22.8,0.592
DeAndre_Jordan_2019,C,DeAndre-Jordan,265,6-11,30.0,69.0,69.0,29.7,0.641,0.0,...,0.705,3.3,13.1,2.3,0.6,1.1,2.2,11.0,15.0,0.674
Thaddeus_Young_2019,F,Thaddeus-Young,235,6-8,30.0,81.0,81.0,30.7,0.527,0.6,...,0.644,2.4,6.5,2.5,1.5,0.4,1.5,12.6,18.0,0.569


In [8]:
df2019.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286 entries, Kevin_Durant_2019 to CJ_Wilcox_2019
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pos     286 non-null    object 
 1   name    286 non-null    object 
 2   weight  280 non-null    object 
 3   height  280 non-null    object 
 4   Age     259 non-null    float64
 5   G       259 non-null    object 
 6   GS      259 non-null    object 
 7   MP      259 non-null    object 
 8   FG%     256 non-null    object 
 9   3P      259 non-null    object 
 10  3P%     244 non-null    object 
 11  FT      259 non-null    object 
 12  FT%     242 non-null    object 
 13  ORB     259 non-null    object 
 14  TRB     259 non-null    object 
 15  AST     259 non-null    object 
 16  STL     259 non-null    object 
 17  BLK     259 non-null    object 
 18  TOV     259 non-null    object 
 19  PTS     259 non-null    object 
 20  USG%    258 non-null    float64
 21  TS%     255 non-n

In [13]:
df2018 = scrape_fa_year(2018)
df2018.head(20)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195


,pos,name,weight,height,Age,G,GS,MP,FG%,3P,...,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS,USG%,TS%
LeBron_James_2018,F-G,LeBron-James,250,6-9,33.0,82.0,82.0,36.9,0.542,1.8,...,0.731,1.2,8.6,9.1,1.4,0.9,4.2,27.5,31.6,0.621
Nikola_Jokic_2018,C-F,Nikola-Jokic,284,6-11,22.0,75.0,73.0,32.6,0.499,1.5,...,0.85,2.6,10.7,6.1,1.2,0.8,2.8,18.5,24.2,0.603
Kevin_Durant_2018,F-G,Kevin-Durant,240,6-10,29.0,68,68,34.2,.516,2.5,...,.889,0.5,6.8,5.4,0.7,1.8,3.0,26.4,30.4,0.640
Clint_Capela_2018,C,Clint-Capela,240,6-10,23.0,74.0,74.0,27.5,0.652,0.0,...,0.56,3.3,10.8,0.9,0.8,1.9,1.4,13.9,19.4,0.650
Chris_Paul_2018,G,Chris-Paul,175,6-0,32.0,58.0,58.0,31.8,0.46,2.5,...,0.919,0.6,5.4,7.9,1.7,0.2,2.2,18.6,24.5,0.604
DeAndre_Jordan_2018,C,DeAndre-Jordan,265,6-11,29.0,77.0,77.0,31.5,0.645,0.0,...,0.58,4.3,15.2,1.5,0.5,0.9,1.8,12.0,15.2,0.648
Paul_George_2018,F,Paul-George,220,6-8,27.0,79.0,79.0,36.6,0.43,3.1,...,0.822,0.9,5.7,3.3,2.0,0.5,2.7,21.9,25.7,0.570
Derrick_Favors_2018,F-C,Derrick-Favors,265,6-9,26.0,77.0,77.0,28.0,0.563,0.2,...,0.651,2.6,7.2,1.3,0.7,1.1,1.1,12.3,18.5,0.593
Dwight_Howard_2018,C,Dwight-Howard,265,6-10,32.0,81.0,81.0,30.4,0.555,0.0,...,0.574,3.1,12.5,1.3,0.6,1.6,2.6,16.6,24.2,0.577
Julius_Randle_2018,F-C,Julius-Randle,250,6-8,23.0,82.0,49.0,26.7,0.558,0.1,...,0.718,2.2,8.0,2.6,0.5,0.5,2.6,16.1,25.3,0.606


In [14]:
df2018.info(20)

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, LeBron_James_2018 to Malik_Newman_2018
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pos     186 non-null    object 
 1   name    186 non-null    object 
 2   weight  186 non-null    object 
 3   height  186 non-null    object 
 4   Age     183 non-null    float64
 5   G       183 non-null    object 
 6   GS      183 non-null    object 
 7   MP      183 non-null    object 
 8   FG%     183 non-null    object 
 9   3P      183 non-null    object 
 10  3P%     171 non-null    object 
 11  FT      183 non-null    object 
 12  FT%     174 non-null    object 
 13  ORB     183 non-null    object 
 14  TRB     183 non-null    object 
 15  AST     183 non-null    object 
 16  STL     183 non-null    object 
 17  BLK     183 non-null    object 
 18  TOV     183 non-null    object 
 19  PTS     183 non-null    object 
 20  USG%    182 non-null    float64
 21  TS%     182 no

In [15]:
df2017 = scrape_fa_year(2017)
df2017.head(20)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179


,pos,name,weight,height,Age,G,GS,MP,FG%,3P,...,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS,USG%,TS%
Stephen_Curry_2017,G,Stephen-Curry,185,6-3,28.0,79.0,79.0,33.4,0.468,4.1,...,0.898,0.8,4.5,6.6,1.8,0.2,3.0,25.3,30.1,0.624
Kevin_Durant_2017,F-G,Kevin-Durant,240,6-10,28.0,62,62,33.4,.537,1.9,...,.875,0.6,8.3,4.8,1.1,1.6,2.2,25.1,27.8,0.651
Gordon_Hayward_2017,F-G,Gordon-Hayward,225,6-7,26.0,73.0,73.0,34.5,0.471,2.0,...,0.844,0.7,5.4,3.5,1.0,0.3,1.9,21.9,27.6,0.595
Kyle_Lowry_2017,G,Kyle-Lowry,196,6-0,30.0,60.0,60.0,37.4,0.464,3.2,...,0.819,0.8,4.8,7.0,1.5,0.3,2.9,22.4,24.9,0.623
Otto_Porter_2017,F,Otto-Porter,198,6-8,23.0,80.0,80.0,32.6,0.516,1.9,...,0.832,1.5,6.4,1.5,1.5,0.5,0.6,13.4,15.1,0.628
Jeff_Teague_2017,G,Jeff-Teague,195,6-3,28.0,82.0,82.0,32.4,0.442,1.1,...,0.867,0.4,4.0,7.8,1.2,0.4,2.6,15.3,22.1,0.574
Blake_Griffin_2017,F,Blake-Griffin,250,6-9,27.0,61,61,34.0,.493,0.6,...,.760,1.8,8.1,4.9,0.9,0.4,2.3,21.6,28.0,0.569
Andre_Iguodala_2017,G-F,Andre-Iguodala,215,6-6,33.0,76.0,0.0,26.3,0.528,0.8,...,0.706,0.7,4.0,3.4,1.0,0.5,0.8,7.6,11.2,0.624
Danilo_Gallinari_2017,F,Danilo-Gallinari,233,6-10,28.0,63,63,33.9,.447,2.0,...,.902,0.6,5.2,2.1,0.6,0.2,1.3,18.2,20.0,0.622
Pau_Gasol_2017,C-F,Pau-Gasol,250,7-0,36.0,64.0,39.0,25.4,0.502,0.9,...,0.707,1.7,7.8,2.3,0.4,1.1,1.3,12.4,21.3,0.578


In [18]:
df2017.sample(20)

,pos,name,weight,height,Age,G,GS,MP,FG%,3P,...,FT%,ORB,TRB,AST,STL,BLK,TOV,PTS,USG%,TS%
Amir_Johnson_2017,F-C,Amir-Johnson,240,6-9,29.0,80.0,77.0,20.1,0.576,0.3,...,0.67,1.5,4.6,1.8,0.6,0.8,1.0,6.5,13.6,0.628
Ersan_Ilyasova_2017,F,Ersan-Ilyasova,235,6-9,29.0,82,52,26.1,.431,1.7,...,.778,1.4,5.9,1.7,0.7,0.3,1.4,13.1,22.2,0.546
Rudy_Gay_2017,F,Rudy-Gay,250,6-8,30.0,30.0,30.0,33.8,0.455,1.4,...,0.855,1.2,6.3,2.8,1.5,0.9,2.5,18.7,25.8,0.559
Rajon_Rondo_2017,G,Rajon-Rondo,180,6-1,30.0,69.0,42.0,26.7,0.408,0.7,...,0.6,1.1,5.1,6.7,1.4,0.2,2.4,7.8,17.8,0.461
George_Hill_2017,G,George-Hill,188,6-4,30.0,49.0,49.0,31.5,0.477,1.9,...,0.801,0.5,3.4,4.2,1.0,0.2,1.7,16.9,23.5,0.599
Shane_Larkin_2017,G,Shane-Larkin,175,5-11,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dante_Cunningham_2017,F,Dante-Cunningham,230,6-8,29.0,66.0,35.0,25.0,0.485,1.1,...,0.593,0.8,4.2,0.6,0.6,0.4,0.4,6.6,10.7,0.586
Bojan_Bogdanovic_2017,G-F,Bojan-Bogdanovic,226,6-7,27.0,81,54,25.7,.445,1.8,...,.893,0.5,3.4,1.4,0.4,0.1,1.6,13.7,22.4,0.585
James_Johnson_2017,F,James-Johnson,240,6-7,29.0,76.0,5.0,27.4,0.479,1.1,...,0.707,0.9,4.9,3.6,1.0,1.1,2.3,12.8,22.0,0.564
Trey_Burke_2017,G,Trey-Burke,185,6-0,24.0,57.0,0.0,12.3,0.455,0.5,...,0.759,0.1,0.8,1.8,0.2,0.1,0.8,5.0,19.6,0.532


In [27]:
df2017.to_csv(r'/Users/andrei/Dropbox/Metis/2-Regression/andrei-reg-project/df2017.csv')
df2018.to_csv(r'/Users/andrei/Dropbox/Metis/2-Regression/andrei-reg-project/df2018.csv')
df2019.to_csv(r'/Users/andrei/Dropbox/Metis/2-Regression/andrei-reg-project/df2019.csv')
df2020.to_csv(r'/Users/andrei/Dropbox/Metis/2-Regression/andrei-reg-project/df2020.csv')

In [41]:
df = pd.read_csv('./df2019.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  286 non-null    object 
 1   pos         286 non-null    object 
 2   name        286 non-null    object 
 3   weight      280 non-null    float64
 4   height      280 non-null    object 
 5   Age         259 non-null    float64
 6   G           259 non-null    object 
 7   GS          259 non-null    object 
 8   MP          259 non-null    object 
 9   FG%         256 non-null    object 
 10  3P          259 non-null    object 
 11  3P%         244 non-null    object 
 12  FT          259 non-null    object 
 13  FT%         242 non-null    object 
 14  ORB         259 non-null    object 
 15  TRB         259 non-null    object 
 16  AST         259 non-null    object 
 17  STL         259 non-null    object 
 18  BLK         259 non-null    object 
 19  TOV         259 non-null    o

In [42]:
playerurl = 'https://www.basketball-reference.com/players/f/fournev01.html'
season = 2021
d = scrape_player(playerurl, season)
d['name'] = 'Evan-Fournier'
d['pos'] = 'G-F'
name_year = 'Evan_Fournier_2021'
print(d)

dffournier = pd.DataFrame()
update_row_with_dict(dffournier,d,name_year)

dffournier.to_csv(r'/Users/andrei/Dropbox/Metis/2-Regression/andrei-reg-project/dffournier.csv')

{'weight': '205', 'height': '6-7', 'Age': 28.0, 'G': 42.0, 'GS': 36.0, 'MP': 30.0, 'FG%': 0.457, '3P': 2.8, '3P%': 0.413, 'FT': 2.6, 'FT%': 0.788, 'ORB': 0.2, 'TRB': 3.0, 'AST': 3.4, 'STL': 1.1, 'BLK': 0.5, 'TOV': 1.7, 'PTS': 17.1, 'USG%': 23.1, 'TS%': 0.598, 'name': 'Evan-Fournier', 'pos': 'G-F'}


In [48]:
def get_salaries(salary_url):
    response = requests.get(salary_url)
    page = response.text
    salarysoup = BeautifulSoup(page, "lxml")

    table = salarysoup.find('table')
    table_rows = table.find_all('tr')

    l = []
    for tr in table_rows:
        td = tr.find_all('td')
        row = [tr.text.strip() for tr in td]
        l.append(row)

    df = pd.DataFrame(l)
    df = df.iloc[:,[1,2]]
    return df

dfsal2020 = get_salaries('https://hoopshype.com/salaries/players/')
dfsal2019 = get_salaries('https://hoopshype.com/salaries/players/2019-2020/')
dfsal2018 = get_salaries('https://hoopshype.com/salaries/players/2018-2019/')
dfsal2017 = get_salaries('https://hoopshype.com/salaries/players/2017-2018/')
dfsal2016 = get_salaries('https://hoopshype.com/salaries/players/2016-2017/')

In [54]:
dfsal2020.to_csv(r'/Users/andrei/Dropbox/Metis/2-Regression/andrei-reg-project/dfsal2020.csv')
dfsal2019.to_csv(r'/Users/andrei/Dropbox/Metis/2-Regression/andrei-reg-project/dfsal2019.csv')
dfsal2018.to_csv(r'/Users/andrei/Dropbox/Metis/2-Regression/andrei-reg-project/dfsal2018.csv')
dfsal2017.to_csv(r'/Users/andrei/Dropbox/Metis/2-Regression/andrei-reg-project/dfsal2017.csv')
dfsal2016.to_csv(r'/Users/andrei/Dropbox/Metis/2-Regression/andrei-reg-project/dfsal2016.csv')

df = pd.read_csv('./dfsal2017.csv')
df.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  587 non-null    int64 
 1   1           587 non-null    object
 2   2           587 non-null    object
dtypes: int64(1), object(2)
memory usage: 13.9+ KB


In [55]:
df.sample(10)

,Unnamed: 0,1,2
339,339,Jarrett Allen,"$1,713,720"
96,96,Cody Zeller,"$12,584,270"
202,202,Patrick Patterson,"$5,192,000"
496,496,Jeremy Evans,"$154,059"
299,299,Udonis Haslem,"$2,328,652"
520,520,Marcus Paige,"$77,250"
184,184,Brandan Wright,"$5,955,760"
32,32,Giannis Antetokounmpo,"$22,471,911"
183,183,Alan Williams,"$6,000,000"
573,573,Michael Young,"$77,250"


In [43]:
salary_url = 'https://hoopshype.com/salaries/players/'
response = requests.get(salary_url)
page = response.text
salarysoup = BeautifulSoup(page, "lxml")

table = salarysoup.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    l.append(row)
    
df = pd.DataFrame(l)
df = df.iloc[:,[1,2]]
df


,1,2
0,Player,2020/21
1,Stephen Curry,"$43,006,362"
2,Chris Paul,"$41,358,814"
3,Russell Westbrook,"$41,358,814"
4,James Harden,"$41,254,920"
...,...,...
578,Tyler Hall,"$25,000"
579,Andrew White,"$25,000"
580,Elijah Bryant,"$24,611"
581,Caleb Homesley,"$18,458"


In [44]:
salary_url = 'https://hoopshype.com/salaries/players/2019-2020/'
response = requests.get(salary_url)
page = response.text
salarysoup = BeautifulSoup(page, "lxml")

table = salarysoup.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    l.append(row)
    
pd.DataFrame(l)
df = pd.DataFrame(l)
df = df.iloc[:,[1,2]]
df

,1,2
0,Player,2019/20
1,Stephen Curry,"$40,231,758"
2,Russell Westbrook,"$38,506,482"
3,Chris Paul,"$38,506,482"
4,James Harden,"$38,199,000"
...,...,...
508,Paul Watson,"$79,568"
509,Jarrell Brantley,"$79,568"
510,Justin Wright-Foreman,"$79,568"
511,Garrison Mathews,"$79,568"
